In [1]:
import re
import json
from parsivar import FindStems

## Read files (only for test)

In [2]:
def read_json(path):
    with open(path, 'r') as file:
        return json.load(file)

In [3]:
data = read_json("IR_data_news_12k.json")

In [4]:
content , url , title  = {} , {} , {} 

for i, item in data.items():
    content[i] = item.get('content')
    url[i] = item.get('url')
    title[i] = item.get('title')
    

## Normalizing

### 1. Replace special tokens with placeholders(Email and ID)

In [5]:
def extract_special_tokens(string):
    emails = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', string)
    ids = re.findall(r'@[a-zA-Z0-9_]+', string)
    if ids.count('gmail')> 0:
        ids.remove('@gmail')
    return emails, ids

In [6]:
def replace_special_tokens(string, emails, ids):
    for i, email in enumerate(emails):
        string = string.replace(email, f"__EMAIL_{i}__")
    for i, user_id in enumerate(ids):
        string = string.replace(user_id, f"__ID_{i}__")
    return string

def restore_special_tokens(string, emails, ids):
    for i, email in enumerate(emails):
        string = string.replace(f"__EMAIL_{i}__", email)
    for i, user_id in enumerate(ids):
        string = string.replace(f"__ID_{i}__", user_id)
    
    string = list(string)
    index = 0
    for char in string:
        if char== '_':
            string[index] = ' '
        index += 1
    string = "".join(string)
    return string

In [7]:
text = "ایمیل من example123@gmail.com است و آیدی من @user_name123."
result = extract_special_tokens(text)
print(result)
result = replace_special_tokens(text , result[0] , result[1])
print(result)
result = restore_special_tokens(text , result[0] , result[1])
print(result)

(['example123@gmail.com'], ['@gmail', '@user_name123'])
ایمیل من __EMAIL_0__ است و آیدی من __ID_1__.
ایمیل من example123@gmail.com است و آیدی من @user name123.


### 2. Punctuation removal

In [8]:
def remove_punctoations(string):
    punctoations = ['@', '#', '$', '%', '^', '*', '(', ')', '-', '=', '+', '[', ']', '{', '}', ';', ':', ',', '.',
            '?', '/', '>', '<', '،', '«', '»', '؛', '؟', '"', '!', '!', '!', '"', "'"]

    string = list(string)
    index = 0
    for char in string:
        if char in punctoations:
            string[index] = ' '
        index += 1
    string = "".join(string)

    return string

In [9]:
print(list(content.values())[1])
txt = remove_punctoations(list(content.values())[1])
print(txt)


به گزارش خبرگزاری فارس، سید حمید سجادی در حاشیه مراسم گرامیداشت روز جوان در جمع خبرنگاران در رابطه با عرضه سهام سرخابی‌ها در بورس اظهار داشت: منتظر طی روند هستیم و بعدا اطلاع رسانی خواهیم کرد. وی در مورد حضور تماشاگران در مسابقات فوتبال اظهار داشت: حضور تماشاگران در  لیگ برتر فوتبال تابع نظر فدراسیون ،سازمان لیگ و ستاد ملی مبارزه با کرونا است. انتهای پیام/




به گزارش خبرگزاری فارس  سید حمید سجادی در حاشیه مراسم گرامیداشت روز جوان در جمع خبرنگاران در رابطه با عرضه سهام سرخابی‌ها در بورس اظهار داشت  منتظر طی روند هستیم و بعدا اطلاع رسانی خواهیم کرد  وی در مورد حضور تماشاگران در مسابقات فوتبال اظهار داشت  حضور تماشاگران در  لیگ برتر فوتبال تابع نظر فدراسیون  سازمان لیگ و ستاد ملی مبارزه با کرونا است  انتهای پیام 





### 3. Unicode and number replacements

In [10]:
def change_unicode(string):
    substitutions = {
        # Numbers
        r'(?<!__)0(?!__)': '۰',
        r'(?<!__)1(?!__)': '۱',
        r'(?<!__)2(?!__)': '۲',
        r'(?<!__)3(?!__)': '۳',
        r'(?<!__)4(?!__)': '۴',
        r'(?<!__)5(?!__)': '۵',
        r'(?<!__)6(?!__)': '۶',
        r'(?<!__)7(?!__)': '۷',
        r'(?<!__)8(?!__)': '۸',
        r'(?<!__)9(?!__)': '۹',
        # Arabic and Persian character variations
        r'ﻲ|ﯾ|ﯿ|ي': 'ی',
        r'ﻚ|ﮏ|ﻛ|ﮑ|ﮐ|ك': 'ک',
        r'ء|ې|ێ': 'ی',
        r'ٲ|ٱ|إ|ﺍ|أ|آ': 'ا',
        r'ﺆ|ۊ|ۇ|ۉ|ﻮ|ؤ': 'و',
        r"[ًٌٍَُِّْ]":'',
        r'ّ': '',  # Remove tashdid
        # Special symbols and phrases
        r'﷽': 'بسم االله الرحمن الرحیم',
        # Common substitutions
        r'طهران': 'تهران',
        r'گفت‌وگو|گفت و گو|گفت‌و‌گو': 'گفتگو',
        r'جست‌وجو|جست و جو|جست‌و‌جو': 'جستجو',
        r'طوس': 'توس',
        r'دشک': 'تشک',
        r'﷼': r'ریال',
        r'باطری': 'باتری',
        r'توفان': 'طوفان',
        r'بلیط': 'بلیت',
        r'ﷴ': 'محمد',

        # English to Persian
        r'FIFA': 'فیفا',
    }
    
    # Apply all substitutions
    for pattern, replacement in substitutions.items():
        string = re.sub(pattern, replacement, string)
    
    return string


In [11]:
txt ="ﻲ 123 طهران گفت‌وگو ﷽ دشک FIFA"
txt = change_unicode(txt)
print(txt)

ی ۱۲۳ تهران گفتگو بسم االله الرحمن الرحیم تشک فیفا


### 4. Half-space transformations

In [12]:
def transform_space_to_half_space(string):
    patterns = [
        (r'^(می|نمی)( )', r'\1‌'),  # Prefix "می" or "نمی" at the beginning of the string
        (r'( )(می|نمی)( )', r'\1\2‌'),  # Prefix "می" or "نمی" within the string
        (r'( )(های|ها|هایی|ی|ای|تر|تری|ترین|گر|گری|ام|ات|اش)( )', r'‌\2\3'),  # Common Persian suffixes
        (r'( )(شده|نشده)', r'‌\2'),
        (r'( )(خواهیم|نمی‌خواهیم|می‌خواهند|نمی‌خواهند|می‌خواهید|نمی‌خواهید|می‌خواهیم|نمی‌خواهیم|می‌خواهی|نمی‌خواهی|می‌خواهد|نمی‌خواهد|می‌خواهم|نمی‌خواهم)( )', r'‌\2‌'),  # Complex verb forms
    ]
    
    result = string
    for pattern, replacement in patterns:
        result = re.sub(pattern, replacement, result)
    
    return result

In [13]:
i = 1
print(list(content.values())[i])
txt = transform_space_to_half_space(list(content.values())[i])
print(txt)


به گزارش خبرگزاری فارس، سید حمید سجادی در حاشیه مراسم گرامیداشت روز جوان در جمع خبرنگاران در رابطه با عرضه سهام سرخابی‌ها در بورس اظهار داشت: منتظر طی روند هستیم و بعدا اطلاع رسانی خواهیم کرد. وی در مورد حضور تماشاگران در مسابقات فوتبال اظهار داشت: حضور تماشاگران در  لیگ برتر فوتبال تابع نظر فدراسیون ،سازمان لیگ و ستاد ملی مبارزه با کرونا است. انتهای پیام/




به گزارش خبرگزاری فارس، سید حمید سجادی در حاشیه مراسم گرامیداشت روز جوان در جمع خبرنگاران در رابطه با عرضه سهام سرخابی‌ها در بورس اظهار داشت: منتظر طی روند هستیم و بعدا اطلاع رسانی‌خواهیم‌کرد. وی در مورد حضور تماشاگران در مسابقات فوتبال اظهار داشت: حضور تماشاگران در  لیگ برتر فوتبال تابع نظر فدراسیون ،سازمان لیگ و ستاد ملی مبارزه با کرونا است. انتهای پیام/





### 5. Separate numbers from text

In [14]:
def separate_numbers(string):
    find1 = r'([0-9۰-۹]+)'
    replace1 = r' \1 '
    result1 = re.sub(find1, replace1, string)
    return result1

In [15]:
txt = text = "I have5 books and۳ notebooks."
txt = separate_numbers(txt)
print(txt)

I have 5  books and ۳  notebooks.


### 6. Stemming 

In [16]:
def stemming(string):
    
    string = list(string)
    index = 0
    for char in string:
        if char == '&':
            string[index] = ' '
        index += 1
    string = "".join(string)

    stemmer = FindStems()
    strings = string.split()

    index = 0
    for word in strings:
        if word == 'است': 
            continue
        strings[index] = stemmer.convert_to_stem(word)
        index += 1
    string = " ".join(strings)
    return string

In [17]:
i = 0
print(list(content.values())[i])
txt = stemming(list(content.values())[i])
print(txt)


به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/



به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان قرعه کشید&کش جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشید&کش جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته

## Normalizer

In [18]:
def normalizer(string):
    emails, ids = extract_special_tokens(string)
    string = replace_special_tokens(string, emails, ids)
    string = remove_punctoations(string)
    string = change_unicode(string)
    string = transform_space_to_half_space(string)
    string = stemming(string)
    string = separate_numbers(string)
    string = restore_special_tokens(string, emails, ids)

    return string

In [19]:
i = 0
print(list(content.values())[i])
txt = normalizer(list(content.values())[i])
print(txt)


به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/



به گزارش خبرگزاری فارس کنفدراسیون فوتبال اسیا AFC در نامه رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان قرعه کشید&کش جام باشگاه فوتسال اسیا را رسما اعلام کرد بر این اساس  ۲۵  فروردین ماه  ۱۴۰۱  مراسم قرعه کشید&کش جام باشگاه فوتسال اسیا در مالزی برگزار شد&شو باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال  ۱۴۰۰  به این مسابقات راه پیدا کرده پیش از این گیتی پسند تجربه  ۳  دوره حضور در جام باشگاه فوتسال اسیا را داشته که هر سه دوره به ف

## Tokenizer

In [20]:
def tokenizer(string):
    string = normalizer(string)
    tokens = re.split(r'\s+', string)
    # The \s (lowercase s ) matches a whitespace (blank, tab \t , and newline \r or \n )
    return tokens

In [21]:
i = 0
print(list(content.values())[i])
tokens = tokenizer(list(content.values())[i])
print()
for i in range (len(tokens)):
    print(tokens[i])


به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/




به
گزارش
خبرگزاری
فارس
کنفدراسیون
فوتبال
اسیا
AFC
در
نامه
رسمی
به
فدراسیون
فوتبال
ایران
و
باشگاه
گیتی
پسند
زمان
قرعه
کشید&کش
جام
باشگاه
فوتسال
اسیا
را
رسما
اعلام
کرد
بر
این
اساس
۲۵
فروردین
ماه
۱۴۰۱
مراسم
قرعه
کشید&کش
جام
باشگاه
فوتسال
اسیا
در
مالزی
برگزار
شد&شو
باشگاه
گیتی
پسند
بعنوان
قهرمان
فوتسال
ایران
در
سال
۱۴۰۰
به
این
مسابقات
راه
پیدا
کرده
پیش
از
این
گیتی
پسند
تجربه
۳
دوره
حضور
در
جام
باشگاه
فوتسال
اسیا
را
داشته
که
هر
سه
دوره
به
فینال
مس